### Ez a File kell, hogy megnézzük, előre, hogyan tud tervezni, illetve, hogy a Kalman Filtert kipróbáljuk

In [1]:
import carla
import numpy as np
import cv2
import time
import sys
sys.path.append('F:\CARLA\Windows\CARLA_0.9.15\PythonAPI\carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner
import matplotlib.pyplot as plt
from scipy.interpolate import splprep, splev, interp1d
import math

In [2]:
client = carla.Client('localhost', 2000)

world = client.get_world()

blueprints = [bp for bp in world.get_blueprint_library().filter('*')]
spawn_points = world.get_map().get_spawn_points()

In [3]:
def carorigin():
    global vehicle, vehicle_start_point
    vehicle_bp = world.get_blueprint_library().filter('*mini*')
    vehicle_start_point = spawn_points[94]
    vehicle = world.try_spawn_actor(vehicle_bp[0], vehicle_start_point)

In [4]:
def bicycleorigin():
    global bicycle
    bicycle_bp = world.get_blueprint_library().filter('*crossbike*')
    bicycle_start_point = spawn_points[1]

    bicycle = world.try_spawn_actor(bicycle_bp[0], bicycle_start_point)
    bicyclepos = carla.Transform(bicycle_start_point.location + carla.Location(x=-3, y=3.5))
    bicycle.set_transform(bicyclepos)

In [5]:
def destroy():#Destroying the existing things
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()

In [20]:
vehicle = None
bicycle = None
vehicle_start_point = None
destroy()
#bicycleorigin()
carorigin()

In [ ]:
#targetid = 27
targetid = 4
targetPoint = spawn_points[targetid]

point_A = vehicle_start_point.location
point_B = targetPoint.location


sampling_resolution = 4
grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)


route = grp.trace_route(point_A, point_B)



for index, waypoint in enumerate(route):
        # Draw a string with an ID at the location of each spawn point
        point_id = f'ID: {index}'
        world.debug.draw_string(
            waypoint[0].transform.location,
            point_id,
            draw_shadow=False,
            color=carla.Color(r=255, g=255, b=255),
            life_time=20,  # Set to 0 to make it persist indefinitely
            persistent_lines=True
        )
        #drawn_points.append(point)

In [18]:
MAX_STEER_DEGREES = 40

def maintain_speed(s: float) -> float:
    
    PREFERRED_SPEED = 30 #km/h
    SPEED_THRESHOLD = 2 #When we close to the required speed, it drops the throttle
    


    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.8
    else:
        return 0.4

In [13]:
def DrawPointsFor30Sec(world, spawn_points):
    drawn_points = []
    for index, waypoint in enumerate(spawn_points):
        # Draw a string with an ID at the location of each spawn point
        point_id = f'ID: {index}'
        point = world.debug.draw_string(
            waypoint.location,
            point_id,
            draw_shadow=False,
            color=carla.Color(r=255, g=255, b=255),
            life_time=30,  # Set to 0 to make it persist indefinitely
            persistent_lines=True
        )
        drawn_points.append(point)

DrawPointsFor30Sec(world, spawn_points)

In [15]:
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

def get_angle(car, wp):
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y


    #vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5


    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y), (car_vector.x, car_vector.y))

    return degrees

#### How will it look?

In the next segment, the path calculation will take place. Wi will locally calculate where will be a car in X time steps into the future - potentially - In the real world it should be done with something like the Kalman Filter, but here, to that it works, we cam use the basic dinamics of the car.

In [16]:
#Ezt kell majd beleimplementálni a tervezésbe

def position_at_time(t, speed, total_length, s_to_u, tck):
    """
    Given time t (in seconds), speed is in km/h, so we need to change it to m/s compute the (x, y) position along the road.
    If the distance traveled exceeds the total road length, the endpoint is returned.
    """
    speedinms = speed * 1000/3600

    s_t = speedinms * t  # distance traveled in meters
    s_t = min(s_t, total_length)  # clamp to the road's total length
    u_val = float(s_to_u(s_t))
    pos = splev(u_val, tck)
    return pos  # returns a tuple (x, y)

### Here we setup that, we can use 

In [19]:
starttime = time.time()
lookforward = 5
currentwaypoint = 5

while currentwaypoint < len(route)-1:
    world.tick()
    
    while vehicle.get_transform().location.distance(route[currentwaypoint][0].transform.location) < 5:
            currentwaypoint += 1
    
    predicted_angle = get_angle(vehicle, route[currentwaypoint][0])
    v = vehicle.get_velocity()
    speed = round(3.6*math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    estimated_throttle = maintain_speed(speed)


    #Setup the steering angle
    if predicted_angle < -300:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 300:
        predicted_angle = predicted_angle - 360
    steering_angle = predicted_angle

    if predicted_angle < -MAX_STEER_DEGREES:
        steering_angle = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steering_angle = MAX_STEER_DEGREES
    
    steering_angle = steering_angle/75


    #Apply the vehicle control to each of the two vehicles
    vehicle.apply_control(carla.VehicleControl(throttle = estimated_throttle, steer = steering_angle))


    X = []
    Y = []

    cal_coute = route[currentwaypoint+1:len(route) - lookforward+currentwaypoint]
    vehicle_pos = vehicle.get_transform()
    carx = vehicle_pos.location.x
    cary = vehicle_pos.location.y
    X.append(carx)
    Y.append(cary)
    for point in cal_coute:
        x = point[0].transform.location.x
        y = point[0].transform.location.y
        X.append(x)
        Y.append(y)


    X = np.array(X)
    Y = np.array(Y)
    unique_indices = np.unique(X, return_index=True)[1]
    x_unique = X[unique_indices]
    y_unique = Y[unique_indices]
    sorted_indices = np.argsort(x_unique)
    x_sorted = x_unique[sorted_indices]
    y_sorted = y_unique[sorted_indices]

    if len(x_sorted) > 3:
        tck, u = splprep([x_sorted, y_sorted], s=0)

        # Generate new interpolated points
        u_new = np.linspace(0, 1, 300)
        x_new, y_new = splev(u_new, tck)

        for index, waypoint in enumerate(zip(x_new, y_new)):
            # Draw a string with an ID at the location of each spawn point
            curveloc = carla.Location(x = waypoint[0], y = waypoint[1], z = 0)
            point_id = f'ID: {index}'
            world.debug.draw_string(
                curveloc,
                point_id,
                draw_shadow=False,
                color=carla.Color(r=0, g=255, b=255),
                life_time=0.1,  # Set to 0 to make it persist indefinitely
                persistent_lines=True
            )


        # -------------------------------------------------------
        # 3. Compute arc length along the spline & build mapping
        # -------------------------------------------------------
        dx = np.diff(x_new)
        dy = np.diff(y_new)
        ds = np.sqrt(dx**2 + dy**2)
        arc_length = np.concatenate(([0], np.cumsum(ds)))
        total_length = arc_length[-1]

        # Create an inverse mapping from arc length to spline parameter u.
        # This function will tell us, for any distance s (in meters), what the corresponding u is.
        s_to_u = interp1d(arc_length, u_new, kind='linear')


    T = 2
    position = position_at_time(T, speed, total_length, s_to_u, tck)
    car_future_pos = carla.Location(x = float(position[0]), y = float(position[1]), z = 0)
    world.debug.draw_point(
                car_future_pos,
                size = 0.3,
                color=carla.Color(r=0, g=0, b=255),
                life_time=0.1,  # Set to 0 to make it persist indefinitely
            )


vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=0.5))
endtime = time.time()
execution_time = endtime - starttime
print(f"Execution time: {execution_time:.6f} seconds")
destroy()
bicycleorigin()
carorigin()
    

IndexError: list index out of range